In [72]:
# %pip install sklearn
import sklearn
import nltk
import numpy as np
import pandas as pd
# %pip install gensim
import gensim

1. Zadanie: uzyć TDIF na opisie kandydata ze slajdu 5 i danych z kaggle 
2. uzyć NER (spay) + keywords (kex)

In [73]:
vacancy_summary = "should have 5-10 years experience. Should have very good experience with Automation using Python + Selenium +Panda (module) + Pytest"
candidates_summary = "I am experienced in computer vision. I am experienced in creating AI models using Python. I have 10 years experience in computer Vision. I have 10 years experience in Python."
candidate_description = "I have 10 years experience in creating AI models using Python. On top of that, I was working with Pytest and Selenium."

In [74]:
df = pd.read_csv('ai_jobs_ds.csv')
df.head()


,title,location,summary,salary
0,newArtificial intelligence security specialist...,"Hyderabad, Telangana+1 location",An artificial intelligence (AI) specialist app...,NaN
1,newEDA Tool Developer,"Bengaluru, Karnataka",We are seeking highly motivated individuals wi...,NaN
2,Junior Project Engineer - Artificial Intellige...,"Pongumoodu, Thiruvananthapuram, Kerala",We are dedicated to providing quality and effe...,"₹15,000 - ₹25,000 a month"
3,Test Lead,"Noida, Uttar Pradesh",Working on over 18 different browser/os/ mobil...,NaN
4,+ Lead Testers,"Hyderabad, Telangana",Total Experience: 6- 8 yrs.\r\nAutomation Test...,NaN


In [75]:
from helpers import remove_non_ascii, to_lowercase, remove_punctuation, replace_numbers, remove_stopwords, stem_words, lemmatize_verbs, normalize

normalized_description = " ".join(normalize(nltk.word_tokenize(candidate_description)))
print(normalized_description)
descriptions = []
descriptions_not_normalized = []
labels = []

for i, row in df.iterrows():
    # print(row["title"])
    description_not_normalized = " ".join(nltk.word_tokenize(row["summary"]))
    descriptions_not_normalized.append(description_not_normalized)
    # normalized_title = normalize(nltk.word_tokenize(row["title"]))
    # normalized_location = normalize(nltk.word_tokenize(row["location"]))
    normalized_summary = normalize(nltk.word_tokenize(row["summary"]))
    description = " ".join(np.concatenate([normalized_summary]))
    descriptions.append(description)
    labels.append(row["title"])

normalized_df_to_train = pd.DataFrame({"description": descriptions, "label": labels})
normalized_df_to_train.head()
df_to_train = pd.DataFrame({"description": descriptions_not_normalized, "label": labels})
df_to_train.head()

ten years experience creating ai models using python top working pytest selenium


,description,label
0,An artificial intelligence ( AI ) specialist a...,newArtificial intelligence security specialist...
1,We are seeking highly motivated individuals wi...,newEDA Tool Developer
2,We are dedicated to providing quality and effe...,Junior Project Engineer - Artificial Intellige...
3,Working on over 18 different browser/os/ mobil...,Test Lead
4,Total Experience : 6- 8 yrs . Automation Testi...,+ Lead Testers


In [76]:
#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

zdania = ["Koty lubią mleko", "Psy lubią wodę"]


tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(zdania)

similarity_matrix = cosine_similarity(tfidf_matrix)

print(similarity_matrix)
similarity_score = similarity_matrix[0][1]

print(similarity_score)


[[1.         0.20199309]
 [0.20199309 1.        ]]
0.20199309249791833


In [77]:
def similarity_for_each_row(df, candidate_description):
    similarity_data = []
    tfidf_vectorizer = TfidfVectorizer()
    for i, row in df.iterrows():
        sentenses = [candidate_description, row["description"]]
        tfidf_matrix = tfidf_vectorizer.fit_transform(sentenses)
        similarity_matrix = cosine_similarity(tfidf_matrix)
        score = similarity_matrix[0][1]
        similarity_data.append(score)
    return pd.DataFrame({"similarity": similarity_data, "description": df["description"], "labels": df["label"]})


result = similarity_for_each_row(df_to_train, candidate_description)
result = result.sort_values(by=["similarity"], ascending=False).head(10)
result.iloc[0]["description"], candidate_description
print(candidate_description)
result


I have 10 years experience in creating AI models using Python. On top of that, I was working with Pytest and Selenium.


,similarity,description,labels
85,0.346601,Should have 5-10 years experience . Should hav...,newPython Automation
527,0.275176,2-5 years of experience in a data science/mode...,Data Science Engineer
785,0.272008,Experience in data science models for credit/r...,Data Scientist – Machine Learning
381,0.264757,7+ years of experience working directly with e...,"Manager, Software Development, Alexa Artificia..."
997,0.253143,Experience working with databases handling a l...,newC# Developer
475,0.248444,Required Technical and Professional Expertise ...,newApplication Developer: zOS
491,0.242285,Candidates with 7- 17 Years of total experienc...,Tech Lead/ Associate Architect/ Architect- Art...
800,0.241154,Data science : 2 years ( Required ) . 4+ years...,Data Scientist
521,0.241128,Experience in working with large data sets usi...,Data Scientist
1062,0.238380,8-10 years of working experience in the data a...,Data Scientist - Business Intelligence


In [78]:
print(normalized_description)
result_normalized = similarity_for_each_row(normalized_df_to_train, normalized_description)
result_normalized.sort_values(by=["similarity"], ascending=False).head(10)

ten years experience creating ai models using python top working pytest selenium


,similarity,description,labels
85,0.398941,five hundred and ten years experience good exp...,newPython Automation
366,0.225419,tavant technologies india pvt ltd required exp...,Data Engineer
785,0.209033,experience data science models creditriskfraud...,Data Scientist – Machine Learning
800,0.190825,data science two years required four years exp...,Data Scientist
521,0.180836,experience working large data sets using distr...,Data Scientist
71,0.172086,help client drive business outcomes using data...,Segment Leader- Data & AI- AI
921,0.172086,help client drive business outcomes using data...,Segment Leader- Data & AI- AI
834,0.160804,experience using business intelligence tools e...,Data Scientist
475,0.155909,required technical professional expertise twen...,newApplication Developer: zOS
985,0.155909,advanced programming skills proficiency r pyth...,Data Analyst


In [4]:
import nltk
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

candidate_description = "I have 10 years experience in creating AI models using Python. On top of that, I was working with Pytest and Selenium."

df = pd.read_csv('ai_jobs_ds.csv')
df.head()

descriptions = []
labels = []

for i, row in df.iterrows():
    description = " ".join(nltk.word_tokenize(row["summary"]))
    descriptions.append(description)
    labels.append(row["title"])

df_to_train = pd.DataFrame({"description": descriptions, "label": labels})
df_to_train.head()


def similarity_for_each_row(df, candidate_description):
    similarity_data = []
    tfidf_vectorizer = TfidfVectorizer()
    for i, row in df.iterrows():
        sentenses = [candidate_description, row["description"]]
        tfidf_matrix = tfidf_vectorizer.fit_transform(sentenses)
        similarity_matrix = cosine_similarity(tfidf_matrix)
        score = similarity_matrix[0][1]
        similarity_data.append(score)
    return pd.DataFrame({"similarity": similarity_data, "description": df["description"], "labels": df["label"]})


result = similarity_for_each_row(df_to_train, candidate_description)
result = result.sort_values(by=["similarity"], ascending=False).head(10)
result.iloc[0]["description"], candidate_description
print(candidate_description)
result


I have 10 years experience in creating AI models using Python. On top of that, I was working with Pytest and Selenium.


,similarity,description,labels
85,0.346601,Should have 5-10 years experience . Should hav...,newPython Automation
527,0.275176,2-5 years of experience in a data science/mode...,Data Science Engineer
785,0.272008,Experience in data science models for credit/r...,Data Scientist – Machine Learning
381,0.264757,7+ years of experience working directly with e...,"Manager, Software Development, Alexa Artificia..."
997,0.253143,Experience working with databases handling a l...,newC# Developer
475,0.248444,Required Technical and Professional Expertise ...,newApplication Developer: zOS
491,0.242285,Candidates with 7- 17 Years of total experienc...,Tech Lead/ Associate Architect/ Architect- Art...
800,0.241154,Data science : 2 years ( Required ) . 4+ years...,Data Scientist
521,0.241128,Experience in working with large data sets usi...,Data Scientist
1062,0.238380,8-10 years of working experience in the data a...,Data Scientist - Business Intelligence
